In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from pandas.compat.pyarrow import pa
import time
import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score
import shapely.geometry
from shapely.geometry import LineString, Polygon

In [71]:
data = pd.read_csv('https://raw.githubusercontent.com/AnikaJha/cola_project_dataset/main/panelist_2nd.csv')
data = data[['Panelist', 'Choice', 'Grams_of_sugar_per_100_ml', 'Price']]
data 

,Panelist,Choice,Grams_of_sugar_per_100_ml,Price
0,636220.0,0.0,7.6,65.0
1,636220.0,0.0,10.9,117.0
2,636220.0,0.0,7.8,117.0
3,636220.0,0.0,2.3,38.0
4,636220.0,0.0,2.3,35.4
...,...,...,...,...
119178,700750.0,1.0,4.8,49.0
119179,700750.0,1.0,4.7,49.0
119180,700750.0,4.0,4.8,49.0
119181,700750.0,4.0,4.7,49.0


In [72]:
data.loc[data['Choice']>1, 'Choice'] = 1
data['Panelist'] = data['Panelist'].astype(int)
data['Choice'] = data['Choice'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119183 entries, 0 to 119182
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Panelist                   119183 non-null  int64  
 1   Choice                     119183 non-null  int64  
 2   Grams_of_sugar_per_100_ml  119183 non-null  float64
 3   Price                      119183 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 3.6 MB


In [73]:
# Create a function to calculate profit
def calculate_profit(u_no_sugar, u_sugar, price_no_sugar, price_sugar, q):
    if (u_sugar >= 0 and u_no_sugar >= 0):
        if (u_sugar == u_no_sugar):
            if (price_sugar >= price_no_sugar):
                return price_sugar * q, 'S'
            else: 
                return price_no_sugar * q, 'NS'
        elif (u_sugar > u_no_sugar):
            return price_sugar * q, 'S'
        elif (u_sugar < u_no_sugar):
            return price_no_sugar * q, 'NS'
    elif (u_sugar >= 0 and u_no_sugar < 0):
        return price_sugar * q, 'S'
    elif (u_sugar < 0 and u_no_sugar >= 0):
        return price_no_sugar * q, 'NS'
    else:
        return 0, '-' #???

In [74]:
panelists = data['Panelist'].unique()
panelists_df = pd.DataFrame(panelists, columns=['Panelist'])

num_iterations = 599

final = []

# Perform the resampling iterations
for n in range(num_iterations):
    # Perform the sampling
    sampled_panelists = panelists_df.sample(frac=0.05, replace=True, random_state=5+n)

    # Subset the dataset to include only the rows corresponding to the selected panelists
    df = data[data['Panelist'].isin(sampled_panelists['Panelist'])]
              
    start_time = time.time()
    print("REGRESSIONS")
    ##REGRESSIONS
    # create a dictionary to store the results of the regressions for each panelist
    regressions = {}

    # get all unique panelists from the dataset
    panelists = df["Panelist"].unique()

    #sugar_zero_panelist = []
    #zero_no_sugar_panelist  = []
    # loop through all the unique panelists
    for panelist in panelists:
        # get the data for the current panelist
        panelist_data = df[df["Panelist"] == panelist]
        
        # split the data into two datasets based on the sugar content
        sugar_data = panelist_data[panelist_data["Grams_of_sugar_per_100_ml"] > 0]
        no_sugar_data = panelist_data[panelist_data["Grams_of_sugar_per_100_ml"] == 0]

        # create two separate linear regression models for products with and without sugar

        if sugar_data.shape[0] > 0:
            sugar_reg = LinearRegression().fit(sugar_data[["Price"]], sugar_data[["Choice"]])
            a0_sugar, a1_sugar = sugar_reg.intercept_[0], sugar_reg.coef_[0][0]
        else:
            a0_sugar, a1_sugar = 0, 0
        
        if no_sugar_data.shape[0] > 0:
            no_sugar_reg = LinearRegression().fit(no_sugar_data[["Price"]], no_sugar_data[["Choice"]])
            a0_no_sugar, a1_no_sugar = no_sugar_reg.intercept_[0], no_sugar_reg.coef_[0][0]
        else:
            a0_no_sugar, a1_no_sugar = 0, 0
        
        # store the results of the regression models
        regressions[panelist] = ((a0_sugar, a1_sugar), (a0_no_sugar, a1_no_sugar))

    print('STEP 2')
    ##STEP 2

    results_df = pd.DataFrame(columns=["Panelist", "Sugar R-squared", "No Sugar R-squared", 
                                        "Sugar Intercept", "Sugar Coefficients",
                                        "No Sugar Intercept", "No Sugar Coefficients"])
    k = 0

    for panelist in regressions:
        sugar_reg, no_sugar_reg = regressions[panelist]
        sugar_r2 = 0
        if sugar_reg[0] != 0:
            sugar_data = df[(df["Panelist"] == panelist) & (df["Grams_of_sugar_per_100_ml"] > 0)]
            if sugar_data.shape[0] > 0:
                sugar_preds = sugar_reg[0] + sugar_reg[1]*sugar_data["Price"]
                sugar_r2 = r2_score(sugar_data["Choice"], sugar_preds)

        sugar_intercept = sugar_reg[0]
        sugar_coef = sugar_reg[1]
      
        #No Sugar
        no_sugar_r2 = 0
        if no_sugar_reg[0] != 0:
            no_sugar_data = df[(df["Panelist"] == panelist) & (df["Grams_of_sugar_per_100_ml"] == 0)]
            if no_sugar_data.shape[0] > 0:  # check for no_sugar_data
                no_sugar_preds = no_sugar_reg[0] + no_sugar_reg[1]*no_sugar_data["Price"]
                no_sugar_r2 = r2_score(no_sugar_data["Choice"], no_sugar_preds)

        no_sugar_intercept = no_sugar_reg[0]
        no_sugar_coef = no_sugar_reg[1]

        new_row = {"Panelist": panelist, "Sugar R-squared": sugar_r2, "No Sugar R-squared": no_sugar_r2,
                    "Sugar Intercept": sugar_intercept, "Sugar Coefficients": sugar_coef, 
                    "No Sugar Intercept": no_sugar_intercept, "No Sugar Coefficients": no_sugar_coef}
        results_df = results_df.append(new_row, ignore_index=True)

        results_df['Sugar Coefficients'] = results_df['Sugar Coefficients'].apply(lambda x: (x.tolist()[0])[0] if type(x) is np.ndarray else x)
        results_df['No Sugar Coefficients'] = results_df['No Sugar Coefficients'].apply(lambda x: (x.tolist()[0])[0] if type(x) is np.ndarray else x)
        results_df['Sugar Intercept'] = results_df['Sugar Intercept'].apply(lambda x: (x.tolist()[0]) if type(x) is np.ndarray else x)
        results_df['No Sugar Intercept'] = results_df['No Sugar Intercept'].apply(lambda x: (x.tolist()[0]) if type(x) is np.ndarray else x)

    results_df = results_df.fillna(0)


    print('COORDINATES')
    ##COORDINATES

    coordinates = {}
    rectangles = {}
    diagonals = {}

    for panelist, (sugar_reg, no_sugar_reg) in regressions.items():
        # get the maximum price at which the consumer is willing to buy sugar product
        sugar_consistent_coords = []
        max_price_cc = 0
        if sugar_reg is not None:
            a0, a1 = sugar_reg
            sugar_data = df[(df["Panelist"] == panelist) & (df["Grams_of_sugar_per_100_ml"] > 0)]
            if a1!=0:
                max_price_cc = (0 - a0) / a1
                max_price_cc = max(max_price_cc, 0)  # enforce a minimum price of 0
                if isinstance(sugar_reg, LinearRegression):
                    sugar_consistent_coords = [(0, 0), (max_price_cc, 0), (max_price_cc, sugar_reg.predict([[max_price_cc]])[0]), (0, sugar_reg.predict([[0]])[0])]
                else:
                    sugar_consistent_coords = [(0, 0), (max_price_cc, 0), (0, max_price_cc), (0, 0)]
        else:
            max_price_cc = 0
            sugar_consistent_coords = None
        
        # get the maximum price at which the consumer is willing to buy no sugar product
        no_sugar_consistent_coords = []
        max_price_z = 0
        if no_sugar_reg is not None:
            a0_1, a1_1 = no_sugar_reg
            no_sugar_data = df[(df["Panelist"] == panelist) & (df["Grams_of_sugar_per_100_ml"] == 0)]
            if a1_1!=0:
                max_price_z = (0 - a0_1) / a1_1
                max_price_z = max(max_price_z, 0)  # enforce a minimum price of 0
                if isinstance(no_sugar_reg, LinearRegression):
                    no_sugar_consistent_coords = [(0, 0), (max_price_z, 0), (max_price_z, no_sugar_reg.predict([[max_price_z]])[0]), (0, no_sugar_reg.predict([[0]])[0])]
            else:
                no_sugar_consistent_coords = [(0, 0), (max_price_z, 0), (0, max_price_z), (0, 0)]
        else:
            max_price_z = 0
            no_sugar_consistent_coords = None
        
        if (max_price_cc != 0) and (max_price_z != 0):  
            # store the results of the coordinates
            bottom_left = (0, 0)
            bottom_right = (max_price_cc, 0)
            top_left = (0, max_price_z)
            top_right = (max_price_cc, max_price_z)

            # create a list of the coordinates for the rectangle
            rect_coords = [bottom_left, bottom_right, top_left,top_right]
            
            rectangle = shapely.geometry.Polygon([bottom_left, bottom_right, top_right, top_left])
            rectangles[panelist] = rectangle

            diagonal = shapely.geometry.LineString([(0, max_price_z), (max_price_cc, 0)])
            diagonals[panelist] = diagonal

            # add the rectangle coordinates to the dictionary
            coordinates[panelist] = {
            "max_price_cc": max_price_cc,
            "max_price_z": max_price_z,
            "rect_coords": rect_coords
    }

    # find intersection points between diagonals and rectangles
    intersection_points = {}
    for panelist, rectangle in rectangles.items():
      
        diagonal = diagonals[panelist]
        intersection = rectangle.intersection(diagonal)
        intersection_points[panelist] = intersection

    # sort rectangles by area
    sorted_rectangles = sorted(rectangles.items(), key=lambda x: x[1].area, reverse=True)
    sort_rectangles = dict(sorted_rectangles)
                          
    for panelist, rectangle in sort_rectangles.copy().items():
        if (rectangle.bounds[2] > 1300) | (rectangle.bounds[3] > 1300):
            del sort_rectangles[panelist]

    intersection_points = {}

    # Iterate over all panelists and their rectangles
    for panelist1, rectangle1 in sort_rectangles.items():
        
        # Get the diagonal line for the current panelist
        diagonal1 = diagonals[panelist1]
        
        # Iterate over all other panelists and their rectangles
        for panelist2, rectangle2 in sort_rectangles.items():
            
            # Skip if it is the same rectangle or if it has already been compared
            if panelist1 == panelist2 or (panelist2, panelist1) in intersection_points:
                continue
            
            # Check for intersection between the two rectangles
            if rectangle1.intersects(rectangle2):
                
                # Get the diagonal line for the other panelist
                diagonal2 = diagonals[panelist2]
                
                # Compute the intersection between the two diagonals
                diagonal_intersection = diagonal1.intersection(diagonal2)
                
                # Compute the intersection between the two rectangles
                rectangle_intersection = rectangle1.intersection(rectangle2)
                
                # Compute the intersection between the rectangle and the opposite diagonal
                opposite_diagonal = LineString([(rectangle2.bounds[0], rectangle2.bounds[3]), 
                                                (rectangle2.bounds[2], rectangle2.bounds[1])])
                opposite_intersection = rectangle1.intersection(opposite_diagonal)
                
                # Store the intersection points in a dictionary
                intersection_points[(panelist1, panelist2)] = {
                    "diagonal_intersection": diagonal_intersection,
                    "rectangle_intersection": rectangle_intersection,
                    "opposite_intersection": opposite_intersection
                }


    intersection_point_set = set()

    for intersection in intersection_points.values():
        diagonal_intersection_coords = list(intersection['diagonal_intersection'].coords)
        rectangle_intersection_coords = list(intersection['rectangle_intersection'].exterior.coords)
        opposite_intersection_coords = list(intersection['opposite_intersection'].coords)

        intersection_point_set.update(diagonal_intersection_coords)
        intersection_point_set.update(rectangle_intersection_coords)
        intersection_point_set.update(opposite_intersection_coords)


    print('UTILITY CALCULATION')
    ##UTILITY CALCULATION
    # Define a function to calculate utility
    def calculate_utility(a0, a1, price):
        return a0 + a1*price
    utilities = {}
    # n = 5000
    # intersection_point_array = np.array(intersection_point_list[:n])
    # intersection_point_array = np.array(intersection_point_list[n-1:])
    intersection_point_list = list(intersection_point_set)
    intersection_point_array = np.array(intersection_point_list)

    # Iterate over a subset of intersection points for this panelist
    for i in range(len(intersection_point_array)):
        price_no_sugar, price_sugar = intersection_point_array[i]

        # Get the intercept and coefficients for all panelists at this intersection point
        no_sugar_intercept_array = results_df['No Sugar Intercept'].values
        no_sugar_coefficients_array = results_df['No Sugar Coefficients'].values
        sugar_intercept_array = results_df['Sugar Intercept'].values
        sugar_coefficients_array = results_df['Sugar Coefficients'].values

        # Calculate the utility for each product category using vectorized NumPy array operations
        u_no_sugar_array = calculate_utility(no_sugar_intercept_array, no_sugar_coefficients_array, price_no_sugar)
        u_sugar_array = calculate_utility(sugar_intercept_array, sugar_coefficients_array, price_sugar)

        # Store the utilities for all panelists at this intersection point using a dictionary comprehension
        utilities.update({(panelist, tuple(intersection_point_array[i])): {
            "u_no_sugar": u_no_sugar_array[j],
            "u_sugar": u_sugar_array[j]
        } for j, panelist in enumerate(results_df['Panelist'])})

    # Convert the utilities dictionary to a pandas dataframe
    utility_df = pd.DataFrame.from_dict(utilities, orient='index')
    utility_df.index.names = ['Panelist', 'Intersection_Point']
    utility_df = utility_df.reset_index()

    del regressions
    del intersection_points
    del utilities

    print("PROFIT")
    ##PROFIT
    init_df = pd.read_csv('https://raw.githubusercontent.com/AnikaJha/cola_project_dataset/main/panelist_2nd.csv')
    q = pd.pivot_table(init_df, values='Choice', index='Panelist', aggfunc=sum).reset_index()
    utility_df = utility_df.merge(q[['Panelist', 'Choice']], on='Panelist')
    utility_df['Profit'],utility_df['Type']  = zip(*(utility_df.apply(lambda row: calculate_profit(row['u_no_sugar'], 
                                                                     row['u_sugar'], row['Intersection_Point'][0], row['Intersection_Point'][1], 
                                                                     row['Choice']), axis=1)))
    utility_df['Profit'] = utility_df['Profit'].round(2)
    utility_df = utility_df[utility_df['Type']!='-']

    print("ALPHA")
    ##ALPHA CALCULATION
    # Calculate the alpha value for each pair of intersection points
    from itertools import combinations
    alpha_list = []
    print(len(utility_df['Intersection_Point'].unique()))
    for p1, p2 in combinations(utility_df['Intersection_Point'].unique(), 2):
        alpha = 0
        sq_p1_ns = utility_df[(utility_df['Intersection_Point'] == p1) & (utility_df['Type'] == 'NS')]['Choice'].sum()
        sq_p1_s = utility_df[(utility_df['Intersection_Point'] == p1) & (utility_df['Type'] == 'S')]['Choice'].sum()
        ns_price1 = p1[0]
        s_price1 = p1[1]
        sq_p2_ns = utility_df[(utility_df['Intersection_Point'] == p2) & (utility_df['Type'] == 'NS')]['Choice'].sum()
        sq_p2_s = utility_df[(utility_df['Intersection_Point'] == p2) & (utility_df['Type'] == 'S')]['Choice'].sum()
        ns_price2 = p2[0]
        s_price2 = p2[1]
        denominator = 0
        denominator = (sq_p2_s * (s_price1 - s_price2) - sq_p1_s * (s_price2 - s_price1) + sq_p1_ns * ns_price2 - sq_p2_ns * ns_price1)
        if denominator != 0:
            numerator = 0
            numerator = (sq_p1_ns * ns_price1 + sq_p1_s * s_price1 - sq_p2_ns * ns_price2)
            alpha = numerator / denominator
        else: alpha = 0
        if (len(alpha_list)%1000==0):
            print(len(alpha_list),end=" ")
        alpha_list.append((p1, p2, alpha))

    # Convert the alpha_list to a DataFrame
    alpha_df = pd.DataFrame(alpha_list, columns=['Intersection_Point1', 'Intersection_Point2', 'Alpha'])
    alpha_df = pd.DataFrame(alpha_df, columns = ['Alpha'])
    alpha_df.sort_values(by='Alpha', inplace=True, ascending=True)
    alpha_df.reset_index(inplace=True, drop=True)
    grouped_df = pd.DataFrame(utility_df.groupby(['Intersection_Point','Type'])['Choice'].sum()).reset_index()
    print(alpha_df)


    print("MAX PRROFIT LIST")
    ##MAX PRROFIT LIST
    max_profit_list = []
    for alpha in alpha_df['Alpha']:
        max_profit = {'Alpha': alpha, 'Total_Profit': 0}
        for intersection_point, group in grouped_df.groupby(['Intersection_Point']):
            ns_choice = group.loc[group['Type'] == 'NS', 'Choice'].values[0]
            s_choice = group.loc[group['Type'] == 'S', 'Choice'].values[0]
            ns_profit = intersection_point[0] * ns_choice
            s_profit = intersection_point[1] * s_choice
            s_taxes = s_profit * alpha
            s_profit_alpha = s_profit * (1-alpha)
            total_profit = ns_profit + s_profit_alpha
            if total_profit > max_profit['Total_Profit']:
                max_profit['Intersection_Point'] = intersection_point
                max_profit['NS_Choice'] = ns_choice
                max_profit['S_Choice'] = s_choice
                max_profit['NS_Profit'] = ns_profit
                max_profit['S_Profit_without_1-alpha'] = s_profit
                max_profit['Sum_of_taxes'] = s_taxes
                max_profit['S_Profit_with_1-alpha'] = s_profit_alpha
                max_profit['Total_Profit'] = total_profit
        if (len(max_profit_list)%1000==0):
            print(len(max_profit_list),end=" ")    
        max_profit_list.append(max_profit)
    max_profit_df = pd.DataFrame(max_profit_list)
    max_profit_df2 = max_profit_df[(max_profit_df['Alpha'] < 1) & (max_profit_df['Alpha'] > 0)]
    del max_profit_list

    utility_df_NS = utility_df[utility_df['Type']=='NS'].copy()
    utility_df_S = utility_df[utility_df['Type']=='S'].copy()
    utility_df_NS = utility_df_NS.groupby('Intersection_Point').sum()[['u_no_sugar']].reset_index()
    utility_df_S = utility_df_S.groupby('Intersection_Point').sum()[['u_sugar']].reset_index()
    merged_df = utility_df_NS.merge(utility_df_S, on='Intersection_Point', how='inner')

    print("SW PROCESS")
    #SW PROCESS
    sw_df = max_profit_df.groupby('Intersection_Point').sum()[['NS_Profit','S_Profit_without_1-alpha', 'Sum_of_taxes']].reset_index()
    utility_profit = []
    utility_profit_df = pd.DataFrame()
    for i in sw_df['Intersection_Point'].unique():
        uprofits = {'Intersection_Point': i, 'No_Sugar_Utilities': 0, 'Sugar_Utilities_without_alpha': 0}
        uprofits['No_Sugar_Utilities'] = utility_df_NS.loc[utility_df_NS['Intersection_Point'] == i,'u_no_sugar'].values[0]
        uprofits['Sugar_Utilities_without_alpha'] = utility_df_S.loc[utility_df_S['Intersection_Point'] == i,'u_sugar'].values[0]
        utility_profit.append(uprofits)
        
    utility_profit_df = pd.DataFrame(utility_profit)
    sw_df2 = max_profit_df2.groupby('Intersection_Point').sum()[['NS_Profit','S_Profit_without_1-alpha', 'Sum_of_taxes']].reset_index()

    utility_profit2 = []
    utility_profit_df2 = pd.DataFrame()
    for i in sw_df2['Intersection_Point'].unique():
        uprofits2 = {'Intersection_Point': i, 'No_Sugar_Utilities': 0, 'Sugar_Utilities_without_alpha': 0}
        uprofits2['No_Sugar_Utilities'] = utility_df_NS.loc[utility_df_NS['Intersection_Point'] == i,'u_no_sugar'].values[0]
        uprofits2['Sugar_Utilities_without_alpha'] = utility_df_S.loc[utility_df_S['Intersection_Point'] == i,'u_sugar'].values[0]
        utility_profit2.append(uprofits2)
        
    utility_profit_df2 = pd.DataFrame(utility_profit2)

    sw_df2[['No_Sugar_Utilities', 'Sugar_Utilities_without_alpha']] = utility_profit_df2[['No_Sugar_Utilities', 'Sugar_Utilities_without_alpha']]
    sw_df2['Social_Welfare'] = sw_df2.sum(numeric_only=True,axis=1).astype(int)

    max_sw2 = sw_df2['Social_Welfare'].max()
    print(n, "MAX SW: ", max_sw2)
    max_sw_index2 = sw_df2['Social_Welfare'].idxmax()
    max_sw_coordinates2 = sw_df2.loc[max_sw_index2, 'Intersection_Point']
    print("Intersection Point:", max_sw_coordinates2)
    ip = sw_df2.loc[max_sw_index2, 'Intersection_Point']
    group2 = pd.DataFrame(max_profit_df2.groupby('Intersection_Point').sum()[['S_Profit_with_1-alpha']].reset_index())
    print(group2)
    s_profit_tax = group2[group2['Intersection_Point']==max_sw_coordinates2]['S_Profit_with_1-alpha']
    print(s_profit_tax)
    print(sw_df2[sw_df2['Social_Welfare'] == max_sw2]['Sum_of_taxes'])

    alpha = sw_df2[sw_df2['Social_Welfare'] == max_sw2]['Sum_of_taxes']/s_profit_tax
    print(n, "Alpha: ", alpha, '/n---------')
    print("--- %s seconds ---" % int((time.time() - start_time)))
    final.append((n, alpha, max_sw2))

res_df = pd.DataFrame(final)

REGRESSIONS
STEP 2
COORDINATES
UTILITY CALCULATION
PROFIT
ALPHA
99
0 1000 2000 3000 4000             Alpha
0    -1626.556304
1    -1253.734297
2     -305.165685
3     -295.363485
4     -238.955428
...           ...
4846   226.233014
4847   325.490304
4848   465.215781
4849   586.394974
4850  2033.253374

[4851 rows x 1 columns]
MAX PRROFIT LIST
0 1000 2000 3000 4000 SW PROCESS
0 MAX SW:  350872074
Intersection Point: (471.0683027285371, 230.173174054431)
                      Intersection_Point  S_Profit_with_1-alpha
0  (471.0683027285371, 230.173174054431)           6.673993e+07
0    6.673993e+07
Name: S_Profit_with_1-alpha, dtype: float64
0    3.704607e+07
Name: Sum_of_taxes, dtype: float64
0 Alpha:  0    0.555081
dtype: float64 /n---------
--- 351 seconds ---
REGRESSIONS
STEP 2
COORDINATES
UTILITY CALCULATION
PROFIT
ALPHA
169
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000               Alpha
0     -19675.072855
1      -1427.461505
2       -548.518659
3 

In [80]:
res_df.columns = ['iteration', 'alpha', 'max_sw']
res_df['alpha'] = res_df['alpha'].astype(float)
res_df

,iteration,alpha,max_sw
0,0,0.555081,350872074
1,1,0.705446,1574822360
2,2,0.789217,485998716
3,3,0.567879,1703650875
4,4,0.217710,1432531811


In [4]:
res_df['alpha'].mean()

0.5670667311589919